## Perform functional connectivity analysis on trial data, eyes open, and eyes closed data for all viable CLBP subjects
#### Input: *stc.mat
####  Output: con, visualization

In [1]:
import os
from pathlib import Path
import sys
import numpy as np
sys.path.append("/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/")
from eeg_toolkit import preprocess, utils
import eeg_toolkit.functional_connectivity as fc

2024-01-08 17:37:25,030 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [2]:
# Settings
times_tup, time_win_path = preprocess.get_time_window(5)

# data paths
data_dir = Path("../../Data")
processed_data_path = data_dir / "Processed Data"
stc_path = data_dir / "Source time courses"
EO_resting_data_path = stc_path / "Eyes Open"
EC_resting_data_path = stc_path / "Eyes Closed"
zscored_epochs_data_path = stc_path / "zscored_Epochs" / time_win_path

# save paths
fc_path = data_dir / "Functional connectivity"
EC_resting_save_path = fc_path / "Eyes Open"
EO_resting_save_path = fc_path / "Eyes Open"
zscored_epochs_save_path = fc_path / "zscored_Epochs" / time_win_path

save_paths = [EC_resting_save_path, EO_resting_save_path, zscored_epochs_save_path]

[os.makedirs(path) for path in save_paths if not os.path.exists(path)]

[-2.5,0.0,2.5]


[]

In [3]:
# viable subjects
sub_ids = utils.import_subs(data_dir, "sub_ids.txt")
sub_ids = sub_ids[7:]  # select only 64ch subs and turn into set
# print(sub_ids)

EXCLUDE_SUBS = [
    # Chronic Pain Subjects
    '018', '024', '030', '031', '033', '034', '037', 
    '038', '040', '045', '049', '052', '054', '057', '058',
    # Control Subjects
    'C5.', 'C6.', 'C7.', 'C8.', 'C9.', 'C12', 'C13',
    'C14', 'C16', 'C18', 'C19', 'C20', 'C21', 'C24', 'C26'
    ]

SKIP_NANS = ['018','021','C1.','C2.','C3.']
# TODO: these are just for the grant figures
sub_ids = [sub for sub in sub_ids if sub not in EXCLUDE_SUBS]
sub_ids = [sub for sub in sub_ids if sub not in SKIP_NANS]
print(sub_ids)

['020', '022', '023', '027', '029', '032', '035', '036', '039', '041', '042', '043', '044', '046', '048', '050', '051', '053', '055', '056', 'C10', 'C11', 'C15', 'C17', 'C22', 'C25', 'C27']


In [4]:
sub_ids_CP = [el for el in sub_ids if el.startswith('0')]
sub_ids_HC = [el for el in sub_ids if el.startswith('C')]

print(f"Chronics: {len(sub_ids_CP)}")
print(f"Controls: {len(sub_ids_HC)}")
print(f"Total: {len(sub_ids)}")

Chronics: 20
Controls: 7
Total: 27


In [5]:
# Get stc only from selected labels
roi_names = [  # Left
    "rostralanteriorcingulate-lh",  # Left Rostral ACC
    "caudalanteriorcingulate-lh",  # Left Caudal ACC
    "postcentral-lh",  # Left S1,
    "insula-lh",
    "superiorfrontal-lh",  # Left Insula, Left DL-PFC,
    "medialorbitofrontal-lh",  # Left Medial-OFC
    # Right
    "rostralanteriorcingulate-rh",  # Right Rostral ACC
    "caudalanteriorcingulate-rh",  # Right Caudal ACC
    "postcentral-rh",  # , Right S1
    "insula-rh",
    "superiorfrontal-rh",  # Right Insula, Right DL-PFC
    "medialorbitofrontal-rh",
]  # Right Medial-OFC

Freq_Bands = {
    # 'delta': [0, 4],
    "theta": [4.0, 8.0],
    "alpha": [8.0, 13.0],
    "beta": [13.0, 30.0],
    "low-gamma": [30.0, 58.5],
    "notch": [58.5, 61.5],
    "high-gamma": [61.5, 100.0],
}

sfreq = 400  # Hz
tmin, tmax = 0.0, 1.5  # exclude the baseline period for connectivity estimation

con_methods = [
    "wpli",
    "dpli",
    "plv",
    # "",
]

conditions = ['Hand NS', 
              'Hand LS',
              'Hand HS', 
              'Back NS',
              'Back LS',
              'Back HS',
              'Eyes Open',
              'Eyes Closed']

In [6]:
# Initialize dictionary to store results
results = {}

for group in [sub_ids_CP, sub_ids_HC]:
    # initialize dictionary for this group
    results[group] = {}
    for sub_id in sub_ids_CP:
        # Initialize dictionary for this subject
        results[group][sub_id] = {}
        
        # First load in STC data and separate epochs by stimulus
        # Epochs
        hand_all_label_ts, back_all_label_ts, hand_all_ratings, back_all_ratings = fc.separate_epochs_by_stim(
        sub_id, processed_data_path, zscored_epochs_data_path
    )
        # Resting state
        # label_ts_EO = utils.unpickle_data(
        #     EO_resting_data_path, f"{sub_id}_EO.pkl"
        # )
        # label_ts_EC = utils.unpickle_data(
        #     EC_resting_data_path, f"{sub_id}_EC.pkl"
        # )
        
        # Unpack label_ts for each site and stimulus level
        label_ts_all = []
        label_ts_all.extend(list(hand_all_label_ts))
        label_ts_all.extend(list(back_all_label_ts))
        # label_ts_all.extend([label_ts_EO, label_ts_EC])
        
        # Compute connectivity for epochs
        for method in con_methods: 
            for label_ts,condition in zip(label_ts_all,conditions):
                if isinstance(label_ts, list):
                    con_epochs = fc.compute_connectivity_epochs(
                        label_ts, roi_names, method, Freq_Bands, tmin, tmax, sfreq
                    )
                    
                    # average epochs within subject first
                    con_epochs_mean = np.mean(con_epochs, axis=1)

                    # Add result to dictionary
                    if condition not in results[sub_id]:
                        results[group][sub_id][condition] = {}
                    results[group][sub_id][condition][method] = con_epochs_mean
                else:   
                    con_eo = fc.compute_connectivity_resting_state(label_ts, roi_names, method, Freq_Bands, sfreq, condition="EO")
                    con_ec = fc.compute_connectivity_resting_state(label_ts, roi_names, method, Freq_Bands, sfreq, condition="EC")

                    # Add result to dictionary
                    if "Eyes Open" not in results[sub_id]:
                        results[group][sub_id]["Eyes Open"] = {}
                    results[group][sub_id]["Eyes Open"][method] = con_eo

                    if "Eyes Closed" not in results[sub_id]:
                        results[group][sub_id]["Eyes Closed"] = {}
                    results[group][sub_id]["Eyes Closed"][method] = con_ec

TypeError: unhashable type: 'list'

In [ ]:
# Initialize dictionary to store average results
avg_results = {}

# Loop over each group, subject, condition, and method
for group in results:
    avg_results[group] = {}
    for subject in results[group]:
        for condition in results[group][subject]:
            for method in results[group][subject][condition]:
                ###############################################################################################

                # If this condition and method hasn't been seen yet, initialize a list
                if condition not in avg_results[group]:
                    avg_results[group][condition] = {}
                if method not in avg_results[group][condition]:
                    avg_results[group][condition][method] = []

                # Append this subject's result
                avg_results[group][condition][method].append(results[group][subject][condition][method])

                # Now compute the average for each condition and method
                avg_results[group][condition][method] = np.mean(avg_results[group][condition][method], axis=1)

                ###############################################################################################
                # Get the averaged result               
                avg_con = avg_results[group][condition][method]

                # Plot averaged data and save
                fc.plot_and_save(avg_con, fc.plot_connectivity_circle, method, Freq_Bands, roi_names, group, condition, fc_path)
                fc.plot_and_save(avg_con, fc.plot_global_connectivity, method, Freq_Bands, roi_names, group, condition, fc_path)
                fc.plot_and_save(avg_con, fc.plot_connectivity, method, Freq_Bands, roi_names, group, condition, fc_path)
                fc.plot_and_save(avg_con, fc.plot_con_matrix, method, Freq_Bands, roi_names, group, condition, fc_path)

                
                



In [ ]:
# Loop over each group, condition, and method
for group in avg_results:
    for condition in avg_results[group]:
        for method in avg_results[group][condition]:
            # Get the averaged results for this combination


In [ ]:
# Plot averaged data
plot_connectivity_circle(con_epochs_array, con_methods, Freq_Bands, roi_names)
plot_global_connectivity(epochs, tmin, tmax, n_connections, con_epochs, Freq_Bands)
plot_connectivity(con_epochs, t_con_max, roi_names, con_methods)
plot_con_matrix(con_data, n_con_methods, connectivity_methods, roi_names, foi)
